In [1]:
# -*- coding:utf-8 -*-
# handwritten digits recognition
# Data: MINIST
# size: 28*28  train-->6w test-->1w

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim

In [2]:
# 超参数
epoches = 3
n_classes = 10
batch_size = 64
lr = 0.001

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# get train&test data
train_dataset = torchvision.datasets.MNIST('/Users/jsheng/PycharmProjects/IJCAI18/assignment1/data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST('/Users/jsheng/PycharmProjects/IJCAI18/assignment1/data/', train=False, transform=transforms.ToTensor(), download=False)
train = Data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=5)
test = Data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=5)

In [5]:
# dnn model
# dnn
class ClassifyDNN(nn.Module):
    def __init__(self, n_classes):
        super(ClassifyDNN, self).__init__()
        self.linear1 = nn.Sequential(
            nn.Linear(28*28, 300),
            nn.BatchNorm1d(300),
            nn.ReLU()
        )
        self.linear2 = nn.Sequential(
            nn.Linear(300, 100),
            nn.BatchNorm1d(100),
            nn.ReLU()
        )
        self.linear3 = nn.Sequential(
            nn.Linear(100, n_classes)
        )
        self.ls = nn.Softmax()
    def forward(self, x):
        x = self.linear1(x.view(x.size(0), -1))
        x = self.linear2(x)
        x = self.linear3(x)
        out = self.ls(x)

        return out

classify = ClassifyDNN(n_classes).to(device)
print(classify)

ClassifyDNN(
  (linear1): Sequential(
    (0): Linear(in_features=784, out_features=300, bias=True)
    (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (linear2): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (linear3): Sequential(
    (0): Linear(in_features=100, out_features=10, bias=True)
  )
  (ls): Softmax()
)


In [6]:
# 损失函数，参数优化算法
criterion = torch.nn.CrossEntropyLoss()  # 交叉熵
optimizer = optim.Adam(params=classify.parameters(), lr=lr)

In [7]:
# train
total_step = len(train)
for epoch in range(epoches):
    for i, (images, targets) in enumerate(train):
        images = images.to(device)
        targets = targets.to(device)
        preds = classify(images)

        loss = criterion(preds, targets)
        loss.backward()
        criterion.zero_grad()
        optimizer.step()

        if (i+1)%100==0:
            print('[{}|{}] step:{}/{} loss:{:.4f}'.format(epoch+1, epoches, i+1, total_step, loss.item()))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1|3] step:100/938 loss:1.6955
[1|3] step:200/938 loss:1.5730
[1|3] step:300/938 loss:1.5830
[1|3] step:400/938 loss:1.5356
[1|3] step:500/938 loss:1.5541
[1|3] step:600/938 loss:1.5119
[1|3] step:700/938 loss:1.5333
[1|3] step:800/938 loss:1.5398
[1|3] step:900/938 loss:1.5392
[2|3] step:100/938 loss:1.4768
[2|3] step:200/938 loss:1.5621
[2|3] step:300/938 loss:1.5360
[2|3] step:400/938 loss:1.5706
[2|3] step:500/938 loss:1.5543
[2|3] step:600/938 loss:1.5393
[2|3] step:700/938 loss:1.5657
[2|3] step:800/938 loss:1.5847
[2|3] step:900/938 loss:1.5393
[3|3] step:100/938 loss:1.5080
[3|3] step:200/938 loss:1.5551
[3|3] step:300/938 loss:1.5705
[3|3] step:400/938 loss:1.5549
[3|3] step:500/938 loss:1.5393
[3|3] step:600/938 loss:1.5393
[3|3] step:700/938 loss:1.5705
[3|3] step:800/938 loss:1.5642
[3|3] step:900/938 loss:1.5237


In [8]:
# test
classify.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = classify(images)
        _, maxIndexes = torch.max(outputs, 1)
        correct += (maxIndexes==labels).sum().item()
        total += labels.size(0)

    print('in 1w test_data correct rate = {:.2f}'.format((correct/total)*100))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


in 1w test_data correct rate = 93.12


In [9]:
# save model
torch.save(classify.state_dict(), '/Users/jsheng/PycharmProjects/IJCAI18/assignment1/model/dnn_classify.pt')

In [10]:
print('done')

done
